## 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.7 MB/s 
     |████████████████████████████████| 163 kB 79.0 MB/s 
     |████████████████████████████████| 7.6 MB 72.3 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.5 MB/s 
     |████████████████████████████████| 115 kB 81.7 MB/s 
     |████████████████████████████████| 212 kB 81.7 MB/s 
     |████████████████████████████████| 127 kB 80.4 MB/s 
     |████████████████████████████████| 115 kB 62.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 4.8 MB/s 
     |████████████████████████████████| 166 kB 60.5 MB/s 
     |████████████████████████████████| 182 kB 76.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 83.0 MB/s 
     |████████████████████████████████| 162 kB 69.2 MB/s 
     |████████████████████████████████| 162 kB 82.8 MB/s 
     |████████████████████████████████| 158 kB 82.5 MB/s 
     |████████████████████████████████| 157 kB 58.1 MB/s 
     |████████████████████████████████| 157 kB 82.5 MB/s 
     |████████████████████████████████| 157 kB 84.3 MB/s 
     |████████████████████████████████| 157 kB 82.4 MB/s 
     |████████████████████████████████| 157 kB 81.8 MB/s 
     |████████████████████████████████| 157 kB 79.0 MB/s 
     |████████████████████████████████| 157 kB 80.8 MB/s 
     |████████████████████████████████| 156 kB 80.4 MB/s 


In [ ]:
!pip install koco

In [4]:
!kaggle datasets download -d minjaechoi99/aihub-groom

 87% 33.0M/38.1M [00:00<00:00, 115MB/s] 
100% 38.1M/38.1M [00:00<00:00, 115MB/s]


In [5]:
!unzip /content/aihub-groom.zip

Archive:  /content/aihub-groom.zip
  inflating: data/Training/talksets-train-1.json  
  inflating: data/Training/talksets-train-2.json  
  inflating: data/Training/talksets-train-3.json  
  inflating: data/Training/talksets-train-4.json  
  inflating: data/Training/talksets-train-5.json  
  inflating: data/Validation/talksets-train-6.json  


In [40]:
import torch
import torch.optim as optim

import numpy as np
from tqdm import tqdm as tqdm
import koco

from transformers import(
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    DefaultDataCollator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_constant_schedule,
    AdamW
)

In [7]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## readjson

In [89]:
from pathlib import Path
import json, re
def read_aihub(path):
    path = Path(path)
    with open(path, 'rb') as f:
        print(f)
        squad_dict = json.load(f)
    texts = []
    ids = []
    for group in squad_dict:
        for passage in group['sentences']:
            id = passage['id']
            text = passage['origin_text']
            text = re.sub('#.*?#', '[UNK]', text)
            intensity = passage['intensity']
            if intensity == 0:
                texts.append(text)
                ids.append(id)

    return texts, ids

In [90]:
import pandas as pd

texts = []
for i in range(1,2):
    file_path =  f'/content/data/Training/talksets-train-{i}.json'
    ttexts, iids = read_aihub(file_path)
    texts += ttexts

"""train_dev = koco.load_dataset('korean-hate-speech', mode='train_dev')

for line in train_dev['train']:
    if line['hate'] == 'none':
        texts.append(line['comments'])

for line in train_dev['dev']:
    if line['hate'] == 'none':
        texts.append(line['comments'])"""

moral_df = pd.DataFrame(list(texts),
                        columns =['texts'])

<_io.BufferedReader name='/content/data/Training/talksets-train-1.json'>


In [91]:
moral_df

,texts
0,쉴드가 아니라 국가가 면제해준거야.
1,짐승입장도 들어봐야지ㅋㅋ
2,[UNK]이 이제 급하긴 한가보네
3,깐부라고 부르기까지 하다니
4,어머니 간 이식 해 주기 싫어서 도망쳤다고...?
...,...
30544,앞뒤의 말이 너무 다른 거 아니야?
30545,왜 헤어지고 싶은데?
30546,여사친이랑 카톡하지 말랬더니 SNS 메시지로 연락하더라
30547,근데 누구냐니까 다른 남자인 친구로 속이더라


In [92]:
from datasets import Dataset

moral_data = Dataset.from_pandas(moral_df)

In [93]:
moral_data

Dataset({
    features: ['texts'],
    num_rows: 30549
})

## tokenize

In [94]:
model_name = 'beomi/KcELECTRA-base-v2022'
model_version = 'groom2team/pj3_classifier_data_add/pytorch_finetuned:v2'
max_length = 128

In [95]:
import wandb
run = wandb.init()
artifact1 = run.use_artifact(model_version, type='model')
artifact_dir1 = artifact1.download()

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.load_state_dict(torch.load(artifact_dir1+'/best_model_at_end/pytorch_model.bin'))

tokenizer = AutoTokenizer.from_pretrained(model_name)

wandb: Downloading large artifact pytorch_finetuned:v8, 489.19MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:0.1
Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were

In [96]:
def tokenizeWithoutLabel(data):
    tokenized_datas = tokenizer(
        data['texts'],
        max_length=max_length,
        padding="max_length",
        truncation="only_second"
    )
    return tokenized_datas

In [97]:
device = torch.device('cuda')
model = model.to(device)

In [98]:
def del_list_index(tokens, indexs):
    out = tokens[:]
    indexs = list(indexs)
    for index in indexs[::-1]:
        if out[index] == '[UNK]':
            continue
        del out[index]
    return out

In [99]:
torch.cuda.empty_cache()

In [100]:
from datasets.utils import disable_progress_bar
disable_progress_bar()

In [101]:
from itertools import combinations
def delete_style_token(text):
    texts = []
    batch_size = 64
    texts.append({'ids': [0], 'texts' : text})
    tokens = tokenizer.encode(text)
    token_indexs = range(1, len(tokens) - 1)
    for n in range(1,4):
        if len(token_indexs) < n - 1:
            continue
        for indexs in combinations(token_indexs, n):
            texts.append({'ids' : indexs, 'texts' : tokenizer.decode(del_list_index(tokens, indexs)[1:-1])})
    line_data = Dataset.from_list(texts)
    line_tokenized_datasets = line_data.map(tokenizeWithoutLabel, batched=True, remove_columns=line_data.column_names, )
    data_collator = DefaultDataCollator(return_tensors="pt")
    line_loader = torch.utils.data.DataLoader(line_tokenized_datasets, batch_size=batch_size,
                                            shuffle=False, collate_fn=data_collator,
                                            num_workers=0)
    max_diff = 0
    max_info = {}
    vanil_prob = 0
    for up_i, id_inputs in enumerate(line_loader):
        inputs = {}
        inputs['input_ids'] = id_inputs['input_ids'].to(device)
        inputs['token_type_ids'] = id_inputs['token_type_ids'].to(device)
        inputs['attention_mask'] = id_inputs['attention_mask'].to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        softmax = logits.softmax(dim=-1)
        for i, prob in enumerate(softmax):
            if up_i == 0 and i == 0:
                if prob[0] < 0.7:
                    return 
                else:
                    vanil_prob = prob
            else:
                if vanil_prob[0] - prob[0] > max_diff:
                    max_info = (texts[up_i*batch_size + i], prob)
                    max_diff = vanil_prob[0] - prob[0]
    if max_diff >= 0.2:
        return {'masked':max_info[0]['texts'], 'original':text}
            

In [102]:
train_dataset = []

In [113]:
for text in tqdm(list(moral_data['texts'])):
    data = delete_style_token(text)
    if data:
        train_dataset.append(data)

100%|██████████| 30549/30549 [2:28:14<00:00,  3.43it/s]


In [114]:
import pandas as pd
result = pd.DataFrame.from_dict(train_dataset)

In [115]:
result.to_csv('train_data.csv', index = False)